In [168]:
import requests
from bs4 import BeautifulSoup
import re

def getRequest(url):
    try:
        return BeautifulSoup(requests.get(url).content, "html.parser")        
    except Exception as e:
        print("The URL failed to return")
        print(getattr(e, 'message', repr(e)))
        
        
url = "https://www.racingaustralia.horse"
main_page_url = "https://www.racingaustralia.horse/home.aspx"

soup = getRequest(main_page_url)

results = [i['href'] for i in soup.select('[href*=Calendar_Results]')]

stateResults = []

for item in results:
    state = url + item
    stateResults.append("%s" % state)
# 
#TODO Review option to use /InteractiveForm/TrackCondition.aspx?State=NSW
#TODO Put a loop here to loop the states
interim = stateResults[0]

statePage = requests.get(interim)
stateSoup = BeautifulSoup(statePage.content, "html.parser")


print(f"This is {interim} page details")


gardens = []
for item in stateSoup.findAll('a'):
    if item.attrs['href'].endswith('Gardens'):
        gardens.append(url + item['href'])
        

resultContent = getRequest(gardens[0])

print(resultContent.find('span').text)
# print(resultContent)

keys = []
values = []

r1 = "https://www.racingaustralia.horse/FreeFields/Results.aspx?Key=2021Jun19,NSW,Rosehill%20Gardens#Race1"
test_data = getRequest(r1)
print(test_data.find('span').text + ", " + test_data.find("td", {"class" : "horse"} ).text)

stats = test_data.find('div', {'class': 'race-venue-bottom'})
for data in stats.findAll('b'):
    keys.append(data.text)

def find_by_label(soup, label):
    return soup.find("b", text=re.compile(label)).next_sibling

for key in keys:
    values.append(find_by_label(stats, key).strip())

track_details = dict(zip(keys, values))

print(track_details)


This is https://www.racingaustralia.horse/FreeFields/Calendar_Results.aspx?State=NSW page details
Saturday, 19 June 2021
Saturday, 19 June 2021, ROYALZEL 
{'Rail Position:': '+7m Entire', 'Dual Track Meeting:': 'N', 'Track Type:': 'Turf', 'Track Condition:': 'Heavy 8', 'Weather:': 'Showers', 'Penetrometer:': '5.98', 'Track Information:': 'Chance of possible late showers', 'Results Last Published:': 'Sun 20-Jun-21 1:06AM AEST'}
